In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.runnables import Runnable
from typing import TypedDict, List, Literal, Dict
from politician_agent import PoliticianAgent
from party_agent import PartyAgent
from langgraph.graph import StateGraph, END
from typing import List, Dict, TypedDict
from politician_agent import PoliticianAgent
from party_agent import PartyAgent


# === Typ stanu ===
class DebateState(TypedDict):
    question: str
    opinions: Dict[str, str]
    current_agent: str
    summary: str


# === Tworzymy aktorów ===
party = PartyAgent(name="Platforma Obywatelska", acronym="PO")

# Dodajemy polityków do partii
party.add_politician("Donald Tusk")
party.add_politician("Ewa Kopacz")
party.add_politician("Adam Bodnar")

# Mapujemy nazwiska do instancji agentów
politician_agents = {p.name: p for p in party.politicians}
politician_names = list(politician_agents.keys())


# === Funkcja: polityk odpowiada ===
def politician_response(state: DebateState) -> DebateState:
    name = state["current_agent"]
    agent = politician_agents[name]
    previous_opinions = "\n".join(f"{name} : {opinion}" for name, opinion in state["opinions"].items())
    input_prompt = (
        f"Pytanie : {state['question']}\n\n\
        Uwzględnij dotychczasowe wypowiedzi i odnieś się do nich: {previous_opinions}"
    )

    opinion = agent.answer_question(input_prompt)
    print(f"Opinia {name}: {opinion}\n")
    state["opinions"][name] = opinion
    return state


# === Funkcja: nadzorca zmienia agenta lub kończy ===
def supervisor_decision(state: DebateState) -> Literal["next_politician", "summarize"]:
    if len(state["opinions"]) < len(politician_agents):
        return "next_politician"
    return "summarize"


# === Funkcja: ustawienie kolejnego agenta ===
def next_politician(state: DebateState) -> DebateState:
    for name in politician_names:
        if name not in state["opinions"]:
            state["current_agent"] = name
            break
    return state


# === Funkcja: podsumowanie przez partię ===
def summarize_party_opinion(state: DebateState) -> DebateState:
    summary = party.formulate_party_stance(state["question"])
    state["summary"] = summary
    return state


# === Budowa grafu ===
graph = StateGraph(DebateState)

graph.add_node("get_opinion", politician_response)
graph.add_node("next_politician", next_politician)
graph.add_node("summarize", summarize_party_opinion)

graph.set_entry_point("next_politician")
graph.add_edge("next_politician", "get_opinion")
graph.add_conditional_edges("get_opinion", supervisor_decision, {
    "next_politician": "next_politician",
    "summarize": "summarize"
})
graph.add_edge("summarize", END)

# === Kompilacja ===
debate_chain = graph.compile()




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Donald Tusk poglądy polityczne'}`


Page: Wybory prezydenckie w Polsce w 2005 roku
Summary: Wybory prezydenckie w Polsce odbyły się na terenie całego kraju oraz w kilkudziesięciu obwodach wyborczych ulokowanych za granicą Polski. W wyniku głosowania czwartym prezydentem III Rzeczypospolitej (a trzecim wybranym w wyborach powszechnych) został Lech Kaczyński, który zastąpił Aleksandra Kwaśniewskiego.
Państwowa Komisja Wyborcza zarejestrowała oficjalnie 16 kandydatów. Na kartach do głosowania znalazło się 14 kandydatów, dwóch kolejnych wykreślono przed datą wyborów. Pierwsza tura wyborów prezydenckich odbyła się 9 października 2005, druga tura, do której przeszli Lech Kaczyński oraz Donald Tusk, odbyła się dwa tygodnie później w dniu 23 października 2005.

Page: Platforma Obywatelska
Summary: Platforma Obywatelska (PO) – polska partia polityczna o charakterze centrowym, założona (początkowo jako stowarzyszenie

In [9]:
question = "Czy należy wprowadzić bezwarunkowy dochód podstawowy?"

initial_state = {
    "question": question,
    "opinions": {},
    "current_agent": ""
}

result = debate_chain.invoke(initial_state)

print("\n=== PODSUMOWANIE STANOWISKA PARTII ===")
print(result["summary"])


Opinia Donald Tusk: Dziękuję za to pytanie, które dotyka ważnego zagadnienia w kontekście polityki społecznej. Osobiście uważam, że bezwarunkowy dochód podstawowy to koncepcja, która zasługuje na poważną dyskusję, zwłaszcza w obliczu rosnących nierówności i trudności, z jakimi borykają się wiele osób w naszym społeczeństwie.

W mojej dotychczasowej pracy na rzecz poprawy jakości życia obywateli, zawsze stawiałem na wartość równości i sprawiedliwości społecznej. Bezwarunkowy dochód podstawowy może być jednym z narzędzi, które pomogą w zapewnieniu minimum egzystencji wszystkim obywatelom, a tym samym w redukcji ubóstwa i wspieraniu osób w trudnej sytuacji.

Jednakże, musimy również pamiętać o kontekście gospodarczym. Jako zwolennik społecznej gospodarki rynkowej, uważam, że wprowadzenie takiego systemu wymagałoby starannego przemyślenia, aby nie osłabić motywacji do pracy oraz nie zniechęcić do podejmowania aktywności zawodowej. Konieczne jest znalezienie równowagi między wsparciem dla n